# Van Deemter in Practice: An Interactive Exploration of Chromatographic Efficiency, Flow Rate, and Particle Size

Christian P. Haas, September 2025

## Overview

This notebook turns the van Deemter equation into a hands-on playground. You’ll build intuition for how eddy diffusion (*A*), longitudinal diffusion (*B/u*), and mass transfer (*C·u*) shape plate height *H* as a function of flow rate *u*—and translate that into practical method-design choices for LC (and GC, where noted).

## Learning Objectives

By the end, you will be able to:

* Interpret the *A*, *B/u*, and *C·u* terms and explain how each dominates in different *u*-regimes.
* Identify the optimal flow rate *u*<sub>opt</sub> and corresponding *H*<sub>min</sub>, and relate *H* to plate number (*N = L/H*) and resolution (*R*<sub>s</sub>).
* Compare particle **size** effects on *A* and *C*, and reason about trade-offs among speed, resolution, and pressure limits.
* Make design decisions (e.g., choose *u*, *L*, and particle size) under realistic constraints such as pressure and extra-column broadening.

## Audience & Prerequisites

* **Who:** Students and practitioners in analytical chemistry, chromatography, or method development.
* **No coding required:** Minimal Python familiarity helps only if you modify the content of the notebook.

**Estimated time:** 25–40 minutes.
**Scope:** LC throughout, with GC notes where diffusion/viscosity dominate differently.
**License:** Content CC BY 4.0; example code MIT.
**Contributions:** Issues and pull requests welcome (see **Contributing** in the repo).


## What are we looking at? Separation quality in a nutshell

Chromatographic **separation quality** depends on two big levers:

- **Selectivity** *(how different the analytes interact with the column)* — determines **how far apart** the **peak centers** elute (retention/relative retention, often written as $\alpha$).  
- **Efficiency** *(band broadening, how much each band spreads while traveling)* — determines **how wide** each **peak** becomes (linked to plate height $H$; narrower peaks = better).

In the sketch below, each column shows two colored bands moving through a column. From left to right, separation improves because either the **selectivity increases** (centers farther apart) and/or the **efficiency** increases (band broadening decreases, peaks narrower). In this notebook we’ll **hold the idea of selectivity in the background** and focus on the second lever: **band broadening** and how the **van Deemter equation** explains it.

<img src="content/01/01-chrom-pencils.png" style="width:60%;">

*Caption:* Separation quality depends on both **selectivity** (spacing between peak centers) and **efficiency** (band broadening, peak width). Adapted from [LibreTexts: *Chromatography*](https://chem.libretexts.org/Courses/BethuneCookman_University/BCU%3A_CH-346_Instrumental_Analysis/Chromatography/Chromatography). 


**Influence of efficiency on chromatograms (and separation quality):**  
Below, see three different $H$ (retention times fixed) to compare an *ideal* (very small $H$), a *normal*, and a *poor* (large $H$) case with three peaks each.

Here, **resolution** $R_s$ simply is a measure for how well two peaks are separated: a deep valley (ideally down to the baseline) = **high resolution** and higher $R_s$; a shallow valley or overlap = **low resolution** or lower $R_s$.  

With column length and flow held constant, **decreasing $H$ narrows the peaks and deepens the valley** (better resolution); increasing $H$ does the opposite.

<img src="content/01/01-chromatograms.png" style="width:60%;">

## Theory — a quick introduction into chromatographic efficiency/band broadening

Chromatographic peaks broaden because molecules spread while traveling through the column. The **van Deemter equation** treats this as a budget of broadening mechanisms and predicts how the **plate height** $H$ (axial band variance per unit length) depends on **linear velocity** $u$:

$$
H(u) \;=\; A \;+\; \frac{B}{u} \;+\; C\,u
$$

- **$A$** — *Eddy diffusion:* different path lengths through a packed bed; ~independent of $u$. Negligible in open-tubular GC.  
- **$B/u$** — *Longitudinal diffusion:* axial diffusion; dominates at **low $u$** and scales with the molecular diffusion coefficient.  
- **$C\,u$** — *Mass-transfer resistance:* finite exchange between phases; dominates at **high $u$** (lumped mobile + stationary contributions).

<img src="content/01/01-van-deemter.png" style="width:60%;">

With the link from $H$ to peak width in mind, let’s now ask **what sets $H$ in the first place** and why two of its contributions change with linear velocity $u$. We’ll take the terms one by one, starting with **eddy diffusion $A$**.

## Eddy diffusion ($A$): many paths through a packed bed

**Idea in one sentence:** in a **packed** column, molecules can take **different path lengths** around particles. Even if they start together, they arrive spread out because some paths are shorter, others longer. This path dispersion contributes a **flow-rate–independent** baseline to band broadening.

Real packed beds can develop **channels**—preferential flow pathways caused by voids, wall effects, or local packing defects. Channels shorten some paths and lengthen others, **widening the distribution of path lengths** and thus increasing the $A$ term.

<img src="content/01/01-eddy-channel.png" style="width:20%;">

Representation of a packed chromatographic column with a channel (arrow). Reproduced from [LibreTexts: *Eddy Diffusion (Multipath) Broadening in Chromatography*](https://chem.libretexts.org/Bookshelves/Analytical_Chemistry/Supplemental_Modules_(Analytical_Chemistry)/Analytical_Sciences_Digital_Library/Courseware/Separation_Science/02_Text/03_Broadening_of_Chromatographic_Peaks/02_Eddy_Diffusion_(Multipath)_Broadening_in_Chromatography).

**Key properties**
- **Independent of $u$:** the geometric spread of path lengths doesn’t disappear when you drive the flow faster or slower. Hence $A$ is the **flat** part in $H(u) = A + \tfrac{B}{u} + C\,u$ (see Figure below).
- **Packing quality matters:** more **uniform packing** → fewer extreme paths → **smaller $A$**.
- **Particle size dependence:** for packed beds, $A$ scales roughly with **particle diameter** $d_p$,
  $$
  A \;\approx\; \lambda\, d_p,
  $$
  where $\lambda$ (≈ 1–2 in many texts) captures packing heterogeneity.
- **Where $A$ vanishes:** in **open-tubular GC** (no packing, only a thin film), there are no alternate paths → $A \approx 0$.

**What increases $A$**
- Poor packing or **voids/channels**; **wall effects** at the tube boundary.
- Large or **nonuniform particle sizes**; damaged frits; vibration during operation.

**What decreases $A$**
- **Smaller $d_p$** (e.g., HPLC → UHPLC); **narrow particle size distribution**.
- Careful slurry packing; good frits; avoiding bed disturbances.

**Rule of thumb for method design**
- You cannot tune $A$ by changing flow during a run; you **choose** $A$ through **hardware and column** decisions (particle size/quality, packing). Once the column is installed and equilibrated, $A$ is essentially fixed.

*Next:* we’ll see why **longitudinal diffusion** ($\tfrac{B}{u}$) **decreases** with increasing flow (molecules have less time to diffuse), and why **mass-transfer resistance** ($C\,u$) **increases** with flow (less time to equilibrate between phases).


## Longitudinal diffusion ($B/u$): spreading along the flow

**Idea in one sentence:** while a band travels down the column, molecules **diffuse forward and backward** along the axis. The longer they linger, the more they spread. This contributes the **$B/u$** term in $H(u)=A+\tfrac{B}{u}+C\,u$—**largest at low $u$**, diminishing as flow increases.

**Where $B$ comes from (intuition)**  
Axial variance grows roughly like **$D_m\,t$** (diffusion coefficient of the solute in the mobile phase times time on-column), while residence time per unit length scales like **$t\sim 1/u$**. Hence the plate-height contribution behaves as
$$
H_B \sim \frac{D_m}{u}\quad\Rightarrow\quad H_B=\frac{B}{u},\;\; B\propto D_m.
$$

**Visual intuition 1) Brownian motion at a fixed location (no drift)**

**What it shows:** 1000 molecules undergoing 1D diffusion from a narrow packet at $x=0$.  
**Takeaway:** The cloud spreads and the histogram approaches a **Gaussian** with
$$
\sigma^2(t)=\sigma_0^2+2\,D_m\,t,
$$
so higher $D_m$ $\Rightarrow$ faster broadening.

<div style="display:grid;width:75%;grid-template-columns:repeat(2,1fr);gap:10px;align-items:center">
  <img src="content/01/01-brownian-spread-high-D.gif" style="width:95%;height:95%;">
  <img src="content/01/01-brownian-spread-low-D.gif" style="width:95%;height:95%;">
</div>

*Caption:* Brownian spreading without drift. Theoretical overlay: $\mathcal{N}\!\big(0,\sigma^2(t)\big)$.

 **Visual intuition 2) Band broadening while traveling to the detector (drift $+$ diffusion)**

**What it shows:** A Gaussian band moves along a column of length $L$ with linear velocity $u$ while broadening. The displayed profile conserves **area**; height adjusts as $\sigma_z(t)$ grows.  
**Link to van Deemter:** with $B=\beta\,D_m$,
$$
H_B \;=\; \frac{B}{u} \;=\; \frac{\beta\,D_m}{u}.
$$
Thus **higher $u$** reduces $H_B$ (less time to diffuse), while **higher $D_m$** increases $H_B$.

<div style="display:grid;width:75%;grid-template-columns:repeat(2,1fr);gap:10px;align-items:center">
  <img src="content/01/01-bterm-travel-high-u-low-D.gif" style="width:95%;height:95%;">
  <img src="content/01/01-bterm-travel-low-u-low-D.gif" style="width:95%;height:95%;">
  <img src="content/01/01-bterm-travel-high-u-high-D.gif" style="width:95%;height:95%;">
  <img src="content/01/01-bterm-travel-low-u-high-D.gif" style="width:95%;height:95%;">
</div>

*Caption:* Band transport and broadening. At fixed $L$, $H_B$ decreases with $u$ and increases with $D_m$.

**Key properties**
- **Dominant at low $u$:** doubling $u$ roughly halves $H_B$.  
- **Depends on $D_m$:** large in GC (gases diffuse fast), smaller in LC but still visible at very low $u$.  
- **Largely independent of particle size** (unlike $A$ and parts of $C$), as it’s a mobile-phase property.

**What increases/decreases $B/u$**
- Increase: lower $u$ (more time), higher $D_m$ (faster diffusion: higher $T$, lower viscosity, etc.).  
- Decrease: higher $u$ (watch $C\,u$ and pressure limits), lower $D_m$ (system-dependent trade-offs).

**Rules of thumb**
- At very low $u$, expect **broad, overlap-prone peaks** dominated by $B/u$.  
- In GC the left side of the van Deemter curve is often **diffusion-dominated**; in LC it still explains the steep initial drop of $H$ as $u$ increases from very small values.

*Next:* we examine **mass-transfer resistance ($C\,u$)**—why pushing the flow too high makes peaks widen because phases can’t equilibrate fast enough.


## Mass-transfer resistance ($C\,u$): not enough time to equilibrate

**Idea in one sentence:** When the band moves faster, analyte molecules have **less time to equilibrate** between the mobile and stationary phases. This “lag” makes peaks **broaden more at high $u$**, giving the **$C\,u$** term in $H(u)=A+\frac{B}{u}+C\,u$.

**Where $C$ comes from (intuition)**  
Molecules alternate between moving with the **mobile phase** and being temporarily “parked” in the **stationary phase**. Finite diffusion means concentration profiles **inside particles/films** and **across the stream** need time to flatten. If the flow is too high (or diffusion too slow), these profiles remain uneven → extra variance.

### Visual intuition (mini-sequence across the phases)

**(a) Flow pushes.** The mobile phase moves; stationary phase does not. A gradient appears across the phases.
  
<img src="content/01/01-cterm-0.png" style="width:40%;">

**(b) Diffusion begins.** Diffusion tries to level the gradient.

<img src="content/01/01-cterm-1.png" style="width:40%;">

**(c) Partial equilibration.** If exchange with the stationary phase is slow, the rear of the band lags → tailing (rate-limiting phase determines the skew).

<img src="content/01/01-cterm-2.png" style="width:40%;">

**(d) Full equilibration.** With enough time (lower $u$), gradients flatten → less broadening and more symmetric peaks.

<img src="content/01/01-cterm-3.png" style="width:40%;">

We often split $C$ into **mobile-phase** and **stationary-phase** contributions:
$$
C \approx C_m + C_s,
\qquad
C_m \propto \frac{d_p^{\,2}}{D_m},
\qquad
C_s \propto \left(\frac{k}{1+k}\right)^{\!2}\frac{d_f^{\,2}}{D_s},
$$
where $d_p$ is particle diameter, $D_m/D_s$ are diffusion coefficients in the mobile/stationary phase, $d_f$ is film (or effective) thickness, and $k$ is the retention factor.

**Takeaway:** bigger particles/films and slower diffusion → **larger $C$**.

**Key properties**
- **Dominant at high $u$:** $H_C=C\,u$ grows linearly with $u$.
- **Geometry matters:** smaller $d_p$ and thinner $d_f$ → shorter diffusion paths → **smaller $C$**.
- **Diffusion helps:** higher $D_m$ and $D_s$ (e.g., higher $T$, lower viscosity) lower $C$.
- **Retention dependence:** larger $k$ increases time in the stationary phase → typically **larger $C_s$**.

**What increases $C\,u$**
- Pushing **flow too high** (even if pressure allows it).  
- **Large particles** or **thick stationary films**.  
- **Low diffusion** (viscous solvents, large analytes, low $T$).  
- High **$k$** (more to equilibrate).

**What decreases $C\,u$**
- **Smaller particles** (HPLC → UHPLC), **core–shell** designs; thinner films.  
- **Higher diffusion** (optimize solvent viscosity/temperature within constraints).  
- Moderate **$k$** via selectivity/solvent changes, when compatible with the separation goal.

**Rule of thumb**  
If increasing $u$ now **widens** peaks, you are on the **right-hand** side of the van Deemter curve where $C\,u$ dominates. Choose **particle size/format** to keep $C$ low *before* increasing $u$ aggressively.


## Putting it all together — why small particles changed practice

**Big picture.**  
The van Deemter relation
$$
H(u)=A+\frac{B}{u}+C\,u
$$
shows how plate height (efficiency) varies with linear velocity $u$. Changing particle size $d_p$ mainly affects
- $A \propto d_p$ (multiple paths in the packed bed),
- $C \approx C_m + C_s$ with $C_m \propto \dfrac{d_p^{2}}{D_m}$ and (for films) $C_s \propto \dfrac{d_f^{2}}{D_s}$ (mass transfer).

**What happens when particles get smaller?**
- The entire $H(u)$ curve **drops** (lower $A$, lower $C$).
- The **minimum flattens**: a broader range of $u$ gives near-optimal efficiency.
- The **optimal velocity** shifts higher:
$$
u_{\text{opt}} \approx \sqrt{\frac{B}{C}}
\quad\Rightarrow\quad
u_{\text{opt}} \propto \frac{1}{d_p}.
$$
- The **minimum plate height** scales roughly with particle size:
$$
H_{\min} \sim \mathcal{O}(d_p).
$$

**Why that matters for everyday HPLC/UPLC.**  
Because the minimum is **flatter**, efficiency is now **insensitive to modest flow-rate changes** around $u_{\text{opt}}$. In modern systems ($2–3 \mu\text{m}$ core–shell or $\sim 1.7–2.1 \mu\text{m}$ fully porous on UHPLC hardware), **flow rate is rarely the most critical tuning parameter** during method development. We typically prioritize **selectivity** (phase, solvent, pH, temperature, gradient shape) and **system dispersion** over fine optimization of flow. Flow is set to a sensible value (pressure- and MS-compatible) and left there unless limits force an adjustment.

<img src="content/01/01-van-deemter-dp.png" style="width:60%;">

*Caption:* Notice how $H_{\min}$ drops, $u_{\text{opt}}$ shifts right, and — most importantly — the **curve flattens**, illustrating why we don’t chase an exact flow-rate optimum anymore.

**Backpressure reality check.**  
Using a Kozeny–Carman style scaling,
$$
\Delta P \propto \frac{\eta\,L\,u}{d_p^{2}} .
$$
Halving $d_p$ raises pressure $\sim 4\times$ at the same $L,u$. Since $u_{\text{opt}}$ also doubles, running at the new optimum pushes $\Delta P$ by $\sim 8\times$ — hence the need for **UHPLC** pumps and low-volume plumbing to exploit small particles.

**Rules of thumb (the UPLC playbook).**
- If $d_p \to \tfrac{1}{2}d_p$:

  $H_{\min}\downarrow \sim 2\times,\quad u_{\text{opt}}\uparrow \sim 2\times,\quad t \sim \dfrac{L}{u}\downarrow \sim 2\times,\quad
  \Delta P\uparrow \sim 8\times.$
- Practical compromise: **halve $L$** and run near the higher $u_{\text{opt}}$:

  $N=\dfrac{L}{H}$ $\approx$ maintained, $t\downarrow \sim 4\times$, $\Delta P\uparrow \sim 4\times$ (often acceptable).

**Core–shell alternative.**  
Superficially porous particles shorten intra-particle paths, giving near-UHPLC efficiency at **lower pressure** than fully porous particles of the same $d_p$. Ideal when hardware tops out at $\sim 400–600$ bar.

**Practical checklist (method-dev priorities).**
1. **Selectivity first.** Choose stationary phase and mobile-phase chemistry (pH, modifiers, temperature); no amount of efficiency fixes poor selectivity.
2. **Pick particle and column format** for your pressure budget (1.7–2.1 $\mu$m fully porous on UHPLC; 2.6–3 $\mu$m core–shell for conventional HPLC).
3. **Set column length $L$** by required plates $N=L/H$; smaller $H$ lets you shorten $L$ for speed without losing resolution.
4. **Choose a sensible flow** near $u_{\text{opt}}$ that respects **pressure** and (for LC–MS) **source flow limits**. Don’t micromanage it unless needed — the flattened $H(u)$ makes returns small.
5. **Minimize extra-column dispersion** (short, narrow-ID tubing; low-volume cells; optimized injector and detector settings).
6. **Watch heat and robustness** at high $u$: frictional heating, gradient delay volume, and re-equilibration.
7. **Validate with a test mix** and check peak shapes across the run.

**Take-home message.**  
Smaller particles made the van Deemter minimum **lower and flatter**. That’s why modern HPLC/UPLC can run **faster** with **equal or better resolution**, and why **flow rate is no longer the star knob** — selectivity and system design usually move the needle more.